## Clustering

In [1]:
#from sklearn.cluster import KMeans
from collections import Counter
import math
import pprint
from datetime import datetime
from dateutil.parser import parse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage,fcluster
from scipy import fftpack
from scipy.interpolate import CubicSpline
import time
import json
from fastdtw import fastdtw
import matplotlib.patches as mp
import csv
colormap=np.array(['blue','#d10a3c','#6f7bbb','#3ac467','#c69942','#FF00FF','#DE9816','#87591A','#D473D4','#54F98D','#B3B191'])

import warnings
warnings.filterwarnings('ignore')

### Global fun

In [1]:
def ComputeDtw_Matrix(mat,window=1): #Apply DTW to matrix. Return dissimilarity matrix
    res=np.zeros((mat.shape[0],mat.shape[0]))
    nb=res.shape[0]
    
    for i in range(nb):
        for j in range(i):
            res[i,j]=fastdtw(mat[i],mat[j],window)[0]
            
    return (res+res.T)

def ComputeCurveDerivate_Matrix(DF):#retourne la derivee des series temporelles 
    #r=lignes c=colonnes
    r,c=DF.shape
    #matrice nulle de taille r,
    M_derivate=np.zeros((r,len(np.arange(1,c+1))))
    
    for i in range(0,r):
        cs=CubicSpline(np.arange(1,c+1),DF.values[i])
        M_derivate[i]=cs(np.arange(1,c+1),1)#derivee premiere 

    return pd.DataFrame(M_derivate,columns=np.arange(1,c+1),index=DF.index)

def ComputeDerivativeSpectrum_Matrix(DF): #retourne la matrice des spectres
    r,c=DF.shape
    spectre=np.zeros((r,c))
    
    for i,sensor in enumerate(DF.values):
        #Fast Fourier Transfo        
        spectre[i]=abs(np.fft.fft(sensor))
        
    return spectre


def get_indice_individus(clust):#renvoie la position des individus de chaque cluster
    return [list(np.where(clust==elem)[0]) for elem in np.sort(list(Counter(clust)))]

#def get_individus(clust,data):#renvoie les individus composants chaque cluster
#    return [data.iloc[list(np.where(clust==elem)[0])].values for elem in np.sort(list(Counter(clust)))]

def get_individus(clust, data):
    dic={}
    for i,classe in enumerate(clust):
        if str(classe) not in dic.keys():
            dic[str(classe)]=[]
        dic[str(classe)].append(data.index[i])

    return (dic) 
            
def read_conso_from_csv(file,sep='|'):# consommation to dataframe
    elec2=pd.read_csv(file,sep=sep)
    elec2.columns=["date","prm","puissance"]
    dates=list(elec2.date)
    vec=[x.replace(',','.') for x in elec2['puissance']]
    vec=[x.replace(' ','')for x in vec]
    #2018vec=[x.replace(',','')for x in elec2['puissance']]

    elec2.puissance=[float(x) for x in vec]

    prm_unique=list(np.unique(elec2.prm))
    d_unique=list(np.unique(elec2.date))

    #check if there are missing dates
    res=[]

    #ordre de prm
    for elem in prm_unique:
        tmp=elec2[elec2['prm']==elem][['date','puissance']]
        dd={}
        for i,j in tmp.values:
            dd[i]=j
        res.append(dd)
    for i in range(len(res)):
        if len(res[i]) != len(d_unique):
            missing=np.setxor1d(dates,list(res[i].keys()))
            #intersection
            for j in missing:
                res[i][j]=0
    X= pd.DataFrame(res,index=prm_unique)
    X.columns=pd.to_datetime(X.columns)
    return(X)

def read_conso(file,sep='|'):
    cons=pd.read_csv(file,sep=sep,index_col=0)
    cons.columns=pd.to_datetime(cons.columns)
    return cons

#2018
def read_conso2(file,sep=','):
    cons=pd.read_csv(file,sep=sep,index_col=0)
    cons.columns=pd.to_datetime(cons.columns)
    return cons

def apply_clustering(data,k=2,critere="ward",window=1):
    
    #derivee    
    derivees=ComputeCurveDerivate_Matrix(data)

    #Fourier
    spectres=ComputeDerivativeSpectrum_Matrix(derivees)
    
    #spectre2=np.zeros((239, 365))
    #print(spectres.shape)
    ##print(type(spectres))
    
    #spectre2=pd.DataFrame(spectres)
    
    ###print(spectre2)

    #DTW
    dist=ComputeDtw_Matrix(spectres,window)
    #print(dist)
    
    Z=linkage(dist,critere)
    clus=list(fcluster(Z,k,criterion="maxclust"))
    f=fcluster(Z,k,criterion="maxclust")
    
    return clus,Z,f



#colormap=np.array(['blue','#d10a3c','#6f7bbb','#3ac467','#c69942','#FF00FF','#DE9816','#87591A','#D473D4','#54F98D','#B3B191'])
#colormap=np.array(['#B9648A','#16c72e','#1663c7','#c71f16',"#e8e230","#c76f16","#848484","#f49cc8","#e5d1fa","#5c0a33"])
#colormap=np.array(['blue','#d10a3c','#6f7bbb','#3ac467','#c69942'])
                  
class Inerties():#calcul des inerties à partir des partitions
    def __init__(self,clusters):
        self.gi=np.array([e.mean(axis=0) for e in classes])
        self.g=self.gi.mean(axis=0)
        self.clusters=clusters
        
    def inter(self):
        #norm
        res=[len(self.clusters[i])*math.pow(np.linalg.norm(a-self.g),2) for i,a in enumerate(self.gi)]        
        res=np.sum(res)/len(self.gi)
        return res
    
    def total(self):
        cpt=0
        somme=0
        for c in classes:
            for ind in c:
                somme+=math.pow(np.linalg.norm(ind-self.g),2)        
            cpt+=1
        return somme/cpt         
        
    def intra(self):
        return(self.total()-self.inter())
    
def export_toPredict(prmID,DF):
    data=pd.DataFrame({'ds':DF.loc[prmID].index,'y':DF.loc[prmID]})
    data.to_csv("conso_prediction.csv",sep='|',header=True,index=False)
    print("saved in \'conso_prediction.csv\'")
    
def parse_json(input_file,output_file=None):
    file=json.load(open(input_file))
    buckets=file["aggregations"]["2"]["buckets"]
    
    res={}
    for elem in buckets:
        date=elem['key_as_string']
        tmp={}
        for i in range(len(elem["3"]["buckets"])):
            tmp[elem["3"]["buckets"][i]['key']]=elem["3"]["buckets"][i]['1']['value']
        res[date]=tmp
        
    conso=pd.DataFrame(res)
    conso.columns=pd.to_datetime(conso.columns,format='%d-%m-%Y %H:%M')
    conso.fillna(0,inplace=True)
    
    if output_file:
        conso.to_csv(output_file,sep='|',header=True,index=True)
        print("saved in \'"+output_file+"\'")
    
    return conso

### ELEC2

In [ ]:
# raw CSV n*3
data2=read_conso_from_csv("ELEC-2Cluster_.csv",sep='|')
#2018 data2=read_conso_from_csv("2018_formatted_pipe.csv",sep='|')

In [ ]:
# Raw JSON
data=parse_json("elec_p10_2017_j.json",output_file=None)

In [ ]:
# dfsfd
daily=read_conso2("outnew.csv",sep=',')
#hourly=read_conso("V9_1h.csv",sep='|')

In [ ]:
#print(data.shape)
print(daily.shape)

###  # application

In [ ]:
#days k=2
start=time.time()
cl2,z2,f2=apply_clustering(daily,k=2,window=2)
print("temps d'exécution: ",time.time()-start,'s')
classes=get_individus(cl2,daily)
print()

In [ ]:
#plot
daily.T.plot(legend=False,color=colormap[cl2] ,title='Clustering 2 groupes',figsize=(7,4))
plt.xlabel("Date")
plt.ylabel("Conso(kwh)")

In [ ]:
classes

In [ ]:
#ZOOM
#daily.iloc[:,100:160].T.plot(legend=False,color=colormap[kmeans.labels_],title='Clustering 2 groupes: visualisation sur 30 jours')
#plt.xlabel("Date")
#plt.ylabel("Conso(kW)")

In [ ]:
daily.iloc[:,100:160].T.plot(legend=False,color=colormap[cl2],title='Clustering 2 groupes: visualisation sur 30 jours')
plt.xlabel("Date")
plt.ylabel("Conso(kW)")

In [ ]:
#days k=3
start=time.time()
cl3,z3,f3=apply_clustering(daily,k=3,window=2)
print(time.time()-start)
classes1=get_individus(cl3,daily)
#plot
daily.T.plot(legend=False,color=colormap[cl3] ,title='Clustering 3 groupes',figsize=(7,4))
plt.xlabel("Date")
plt.ylabel("Conso(kwh)")

In [ ]:
classes1

In [ ]:
#days k=4
start=time.time()
cl4,z4,f4=apply_clustering(daily,k=4,window=2)
print(time.time()-start)

#plot
daily.T.plot(legend=False,color=colormap[cl4] ,title='Clustering 4 groupes',figsize=(7,4))
plt.xlabel("Date")
plt.ylabel("Conso(kwh)")

In [ ]:
#### Hourly data

In [ ]:
#Hourly k=2
start=time.time()
clh2,zh2,fh2=apply_clustering(hourly,k=2,window=2)
print(time.time()-start)

#plot
hourly.T.plot(legend=False,color=colormap[clh2] ,title='Clustering 2 groupes',figsize=(7,4))
plt.xlabel("Date")
plt.ylabel("Conso(kwh)")

In [ ]:
#Hourly k=3
start=time.time()
clh3,zh3,fh3=apply_clustering(hourly,k=3,window=2)
print(time.time()-start)

#plot
hourly.T.plot(legend=False,color=colormap[clh3] ,title='Clustering 3 groupes',figsize=(7,4))
plt.xlabel("Date")
plt.ylabel("Conso(kwh)")

In [ ]:
#Hourly k=4
start=time.time()
clh4,zh4,fh4=apply_clustering(hourly,k=4,window=2)
print(time.time()-start)

#plot
hourly.T.plot(legend=False,color=colormap[clh4] ,title='Clustering 4 groupes',figsize=(7,4))
plt.xlabel("Date")
plt.ylabel("Conso(kwh)")

In [ ]:
x=dendrogram(zh2,show_leaf_counts=False)
plt.xlabel("Prm")
plt.ylabel("Distance entre les clusters")

###  # mesures

In [ ]:
classes=get_individus(cl,data)
indices=get_indice_individus(cl)
#Inertie des partitions
print("inertie_inter: {} \nInertie intra: {}".format(Inerties(classes).inter(),Inerties(classes).intra()))

In [ ]:
# generate two clusters: a with 100 points, b with 50:
np.random.seed(4711)  # for repeatability of this tutorial
a = np.random.multivariate_normal([10, 0], [[3, 1], [1, 4]], size=[100,])
b = np.random.multivariate_normal([0, 20], [[3, 1], [1, 4]], size=[50,])
X = np.concatenate((a, b),)
print (X.shape)  # 150 samples with 2 dimensions
plt.scatter(X[:,0], X[:,1])

In [ ]:
Z=linkage(X,'ward')
c, coph_dists = cophenet(Z, pdist(X,'correlation'))
c

In [ ]:
#[idx1, idx2, dist, sample_count]. indique une coupe à 25 pour avoir un bon clustering
plt.plot(sorted(Z[:,2],reverse=True)[:20])
print(Z[-4:,2])

#### Base/load 

In [ ]:
def parse_file_arenh(input_file):
    file=json.load(open(input_file))
    buckets=file["aggregations"]["2"]["buckets"]
    
    res={}
    for elem in buckets:
        date=pd.to_datetime(elem['key'],unit='ms')
        res[date]=elem['3']['buckets'][0]['1']['value']

    return res


In [ ]:
file=parse_file_arenh('dayday_kw.json')#data a la journée
#peak
plt.fill_between(np.arange(len(file.values())),np.sort(list(file.values()))[::-1],color='#9965E3',alpha=.5)
a=np.array(list(file.values()))

#base
min_value=min(a[a!=0])
plt.fill_between(np.arange(len(file.values())),np.repeat(min_value,len(file.values())),color='#0AA240',alpha=.5)
#arenh
res=

#plot
peak=mp.Patch(color='#9965E3',label='peak',alpha=.5)
base=mp.Patch(color='#0AA240',label='base',alpha=.5)
arenh=mp.Patch(color='#E8B426',label='base',alpha=.5)

plt.xlabel('time')
plt.ylabel('puissance (Kw)')
plt.legend(handles=[base,peak])

### Ecart type

In [ ]:
tt=data.loc['30000230557351']
tt.values

In [ ]:
#plot standard deviation
tt=data.loc['30000230557351']

echantillon=tt.values
past=10
error=[0]*past
for i in range(past,len(echantillon)):
    error.append(np.std(echantillon[i-past:i]))

plt.figure(figsize=(14,8))   
plt.errorbar(tt.index,echantillon,error,linestyle='solid',fmt='-o')
plt.title("Standard deviation")
plt.xlabel("date")
plt.ylabel('Puissance KWh')

# cpi

In [2]:
def read_mois(file,sep=';'):    
    #transformation du csv en dataframe
    serie=pd.read_csv(file,sep=sep,index_col=0)
    
    #on remplace les valeurs nulle par des 0 
    #on se debarasse des 2 premiere collonnes dans le dataframe
    #on arrondi les val a 2decimales car CubicSpline attend des val finies
    serie=serie.fillna(0)
    serie=serie.drop(serie.columns[[0,1]], axis='columns')
    serie=round(serie,2)
    return serie

In [ ]:
fic = open('outnewcsv.csv','r')
serie = csv.reader(fic)

data=read_mois(fic,sep=';')
print(data.shape)

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

for k in range(2,11):
    start=time.time()
    cl,z,f=apply_clustering(data,k,window=2)
    print("temps d'exécution: ",time.time()-start,'s')
    classes=get_individus(cl,data)
    print()
    #plot
    data.T.plot(legend=False,color=colormap[cl] ,title='Clustering 2 groupes',figsize=(7,4))
    plt.xlabel("Date")
    plt.ylabel("Conso(kwh)")
    print(k)
    print(silhouette_score(data, cl))
    
x=dendrogram(z,show_leaf_counts=False)
plt.xlabel("")
plt.ylabel("Distance entre les clusters")
print("temps d'exécution: ",time.time()-start,'s')


In [ ]:
fic2 = open('ecoulement_annee_decale.csv','r')
serie1 = csv.reader(fic2)

data2=read_mois(fic2,sep=';')

#on conserve uniquement les 11 premieres annees
data2=data2.iloc[:,0:10]
print(data2.shape)

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

start=time.time()
cl,z,f=apply_clustering(data2,6,window=2)
print("temps d'exécution: ",time.time()-start,'s')
classes=get_individus(cl,data2)
print()
#plot
data2.T.plot(legend=False,color=colormap[cl] ,title='Clustering 6 groupes',figsize=(7,4))
plt.xlabel("Date")
plt.ylabel("%")
print(6)
print(silhouette_score(data2, cl))

In [ ]:
classes

In [ ]:
def returnkey(dic,valsearch):
    res=-1
    for key,value in dic.items():
        if valsearch in value:
            res=key
            break
    return key

In [ ]:
x=dendrogram(z,show_leaf_counts=True)
plt.xlabel("")
plt.ylabel("Distance entre les clusters")
print("temps d'exécution: ",time.time()-start,'s')

### tree

In [15]:
def read_mois2(file,sep=';'):    
    #transformation du csv en dataframe
    serie=pd.read_csv(file,sep=sep,index_col=0)
    
    #on remplace les valeurs nulle par des 0 
    #on se debarasse des 2 premiere collonnes dans le dataframe
    #on arrondi les val a 2decimales car CubicSpline attend des val finies
    serie=serie.fillna(0)
    #serie=serie.drop(serie.columns[0], axis='columns')
    serie=round(serie,2)
    return serie

In [18]:
fic2 = open('ecoulement_annee_decaleed.csv','r')
serie1 = csv.reader(fic2)

data=read_mois2(fic2,sep=';')

#on conserve uniquement les 11 premieres annees
data=data.iloc[:,0:10]
print(data.shape)

(259, 10)


In [19]:
data

,etiquette,affaire,categorie,1,2,3,4,5,6,7
cp in % by month,,,,,,,,,,
12bsmat_nevoy_syst_de_protection_paf,MCI-MCO,0,5,4.94,49.13,8.63,36.14,1.19,0.00,0.00
13_ams_212cp03,MCI-MCO,1,1,71.55,28.32,0.01,0.01,0.00,0.00,0.00
13_cbg_212cp03,MCI-MCO,1,1,46.92,51.52,1.50,0.00,0.00,0.00,0.00
13_ccn_bdd_carcassonne_mc,MCI-MCO,0,1,90.89,8.71,0.00,0.40,0.00,0.00,0.00
13_cfd_bdd_clermont_ferrand_mc,MCI-MCO,0,1,90.96,8.83,0.21,0.00,0.00,0.00,0.00
13_cqv_212cp03,MCI-MCO,1,2,76.89,21.52,1.52,0.00,0.00,0.00,0.00
13_dgn_bdd_draguignan_mc,MCI-MCO,0,1,80.13,19.41,0.30,0.08,0.00,0.08,0.00
13_evx_212cp03,MCI-MCO,1,3,47.64,42.50,9.86,0.00,0.00,0.00,0.00
13_gvc_bdd_gre_chambery_annecy_mc,MCI-MCO,0,3,66.67,31.79,1.32,0.12,0.12,0.00,0.00


In [ ]:
data=data.drop(['categorie'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import jaccard_similarity_score, recall_score
from sklearn.tree import export_graphviz, DecisionTreeClassifier, _tree, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import _tree

from sklearn import tree
from sklearn import metrics
import graphviz

from collections import Counter

from bokeh.plotting import figure

In [ ]:
# split
xtrain, xtest, ytrain, ytest = train_test_split(data.loc[:,data.columns != 'etiquette'],
                                            data['etiquette'],
                                            test_size =0.2,
                                            random_state =42)

print('Original data shape %s' % Counter(ytrain))

In [ ]:
# learn
DR = DecisionTreeClassifier(criterion = "gini", max_depth = 5, random_state=12, min_samples_split=5)
DR = DR.fit(xtrain, ytrain)

In [ ]:
# score
y_pred = DR.predict(xtest)
print(metrics.classification_report(ytest,y_pred))
print(metrics.confusion_matrix(ytest,y_pred))

In [ ]:
import yaml
from bokeh.resources import INLINE
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool,BoxZoomTool, ResetTool, ColumnDataSource

In [ ]:
export_graphviz(DR,out_file="Arbre3.dot",feature_names= xtest.columns,class_names=['1','2','3','4','5','6','7','8','9'],rounded =True,proportion =False,node_ids = True,filled =True)
variables = list(xtest.columns)
counts = DR.feature_importances_
x_range=sorted(variables, key =lambda x: counts[variables.index(x)], reverse=True)
#global feature importance
p = figure(x_range=sorted(variables, key =lambda x: counts[variables.index(x)], reverse=True),plot_height=420,plot_width =1000,title="Features importance")

p.vbar(x = variables, top = counts, width = 0.5)
p.xaxis.major_label_orientation = 0.5
show(p)
    # convert to png
    # dans un terminal lancer :   dot -Tpng DR.dot -o DR.png